sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.   
test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.   
sample_submission.csv - a sample submission file in the correct format.   
items.csv - supplemental information about the items/products.   
item_categories.csv  - supplemental information about the items categories.   
shops.csv- supplemental information about the shops.

教训，将数据类型改为int8会导致负数的出现

<style>
code, kbd, pre, samp {
    font-family:'consolas', Lucida Console, SimSun, Fira Code, Monaco !important;
    font-size: 11pt !important;
}

div.output_area pre {
    font-family: 'consolas', Lucida Console, SimSun, Fira Code, Monaco !important;
    font-size:  10pt !important;
}

div.output_area img, div.output_area svg {
    background-color: #FFFFFF !important;
}
</style>

In [1]:
import numpy as np
import pandas as pd
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
from sklearn.preprocessing import LabelEncoder
import gc
from time_series_pipeline import *
from tqdm import tqdm
from scipy import stats
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import TimeSeriesSplit
from itertools import product

items, shops, cats, train, test = load_data()

items has 22170 rows and 3 columns
shops has 60 rows and 2 columns
cats has 84 rows and 2 columns
train has 2935849 rows and 6 columns
test has 214200 rows and 2 columns
loading data costs 1.31 seconds


In [2]:
df = data_transform(items, shops, cats, train, test)
del items, shops, cats, train
gc.collect()
df

data has 1609122 rows and 4 columns
The program costs 603.98 seconds


,date_block_num,shop_id,item_id,id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_price
0,0,2,19,2_19,0.0,0,40,11,4,NaN
1,0,2,27,2_27,1.0,0,19,5,10,2499.0
2,0,2,28,2_28,0.0,0,30,8,55,NaN
3,0,2,29,2_29,0.0,0,23,5,16,NaN
4,0,2,32,2_32,0.0,0,40,11,4,149.0
...,...,...,...,...,...,...,...,...,...,...
11127999,34,45,18454,45_18454,0.0,20,55,13,2,199.0
11128000,34,45,16188,45_16188,0.0,20,64,14,42,NaN
11128001,34,45,15757,45_15757,0.0,20,55,13,2,199.0
11128002,34,45,19648,45_19648,0.0,20,40,11,4,NaN


In [3]:
df.sort_values(by = ['shop_id','item_id','date_block_num'], inplace = True)
df.reset_index(inplace = True)

start = time.time()

df['lag_1'] = groupby_shift(df, 'item_cnt_month', ['shop_id','item_id'], 1)
df['lag_2'] = groupby_shift(df, 'item_cnt_month', ['shop_id','item_id'], 2)
df['lag_3'] = groupby_shift(df, 'item_cnt_month', ['shop_id','item_id'], 3)
df['lag_6'] = groupby_shift(df, 'item_cnt_month', ['shop_id','item_id'], 6)
df['lag_12'] = groupby_shift(df, 'item_cnt_month', ['shop_id','item_id'], 12)

df['lag_3_roll_avg_3'] = df['lag_3'].rolling(3).mean().astype(np.float32)
df['lag_3_roll_avg_6'] = df['lag_3'].rolling(6).mean().astype(np.float32)
df['lag_12_roll_avg_6'] = df['lag_6'].rolling(6).mean().astype(np.float32)

df['price_shift_1'] = groupby_shift(df, 'item_price', ['shop_id','item_id'], 1)
df['price_shift_2'] = groupby_shift(df, 'item_price', ['shop_id','item_id'], 2)
df['price_shift_3'] = groupby_shift(df, 'item_price', ['shop_id','item_id'], 3)
df['price_shift_6'] = groupby_shift(df, 'item_price', ['shop_id','item_id'], 6)
df['price_shift_12'] = groupby_shift(df, 'item_price', ['shop_id','item_id'], 12)

'''
df['mon_avg_item_cnt'] = groupby_shift(df, 'item_cnt_month', 'date_block_num', 1)
df['mon_avg_item_cnt'] = df.groupby(['date_block_num'])['mon_avg_item_cnt'].transform(lambda x: x.mean())

group = ['date_block_num', 'item_id']
df['mon_item_avg_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_item_avg_1'] = df.groupby(group)['mon_item_avg_1'].transform(lambda x: x.mean())
df['mon_item_avg_2'] = groupby_shift(df, 'item_cnt_month', group, 2)
df['mon_item_avg_2'] = df.groupby(group)['mon_item_avg_2'].transform(lambda x: x.mean())
df['mon_item_avg_6'] = groupby_shift(df, 'item_cnt_month', group, 6)
df['mon_item_avg_6'] = df.groupby(group)['mon_item_avg_6'].transform(lambda x: x.mean())

group = ['date_block_num', 'shop_id']
df['mon_shop_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_shop_1'] = df.groupby(group)['mon_shop_1'].transform(lambda x: x.mean())

group = ['date_block_num', 'shop_id', 'item_category_id']
df['mon_shop_item_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_shop_item_1'] = df.groupby(group)['mon_shop_item_1'].transform(lambda x: x.mean())

group = ['date_block_num', 'shop_id', 'subtype_code']
df['mon_shop_sub_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_shop_sub_1'] = df.groupby(group)['mon_shop_sub_1'].transform(lambda x: x.mean())

group = ['date_block_num', 'city_code']
df['mon_city_avg_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_city_avg_1'] = df.groupby(group)['mon_city_avg_1'].transform(lambda x: x.mean())

group = ['date_block_num', 'item_id', 'city_code']
df['mon_item_city_avg_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_item_city_avg_1'] = df.groupby(group)['mon_item_city_avg_1'].transform(lambda x: x.mean())

group = ['date_block_num', 'type_code']
df['mon_type_avg_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_type_avg_1'] = df.groupby(group)['mon_type_avg_1'].transform(lambda x: x.mean())

group = ['date_block_num', 'subtype_code']
df['mon_subtype_avg_1'] = groupby_shift(df, 'item_cnt_month', group, 1)
df['mon_subtype_avg_1'] = df.groupby(group)['mon_subtype_avg_1'].transform(lambda x: x.mean())
'''
#df['shift6_rolling6_mean'] = df.groupby(['id'])['item_cnt_month'].transform(lambda x: x.shift(6).rolling(6).mean())
#df['shift2_rolling2_mean'] = df.groupby(['id'])['item_cnt_month'].transform(lambda x: x.shift(2).rolling(2).mean())
#df['shift3_rolling1_mean'] = df.groupby(['id'])['item_cnt_month'].transform(lambda x: x.shift(3).rolling(1).mean())   
#df['shift3_rolling2_mean'] = df.groupby(['id'])['item_cnt_month'].transform(lambda x: x.shift(3).rolling(2).mean())
#df['shift6_rolling6_mean'] = df.groupby(['id'])['item_cnt_month'].transform(lambda x: x.shift(6).rolling(6).mean())
print('The program costs %.2f seconds'%(time.time() - start))

The program costs 110.94 seconds


In [4]:
group = df.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_cnt']
group.reset_index(inplace = True)

df = pd.merge(df, group, on = ['date_block_num'], how = 'left')
df['date_cnt'] = df['date_cnt'].astype(np.float16)
df = lag_feature(df, [1], 'date_cnt')
df.drop(['date_cnt'], axis = 1, inplace = True)

group = df.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item']
group.reset_index(inplace = True)

df = pd.merge(df, group, on = ['date_block_num','item_id'], how = 'left')
df['date_item'] = df['date_item'].astype(np.float16)
df = lag_feature(df, [1,2,3,6,12], 'date_item')
df.drop(['date_item'], axis = 1, inplace = True)

group = df.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['date_block_num','shop_id'], how='left')
df['date_shop'] = df['date_shop'].astype(np.float16)
df = lag_feature(df, [1,2,3,6,12], 'date_shop')
df.drop(['date_shop'], axis = 1, inplace = True)

group = df.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_cat']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['date_block_num','item_category_id'], how='left')
df['date_cat'] = df['date_cat'].astype(np.float16)
df = lag_feature(df, [1], 'date_cat')
df.drop(['date_cat'], axis = 1, inplace=True)

In [5]:
df = fill_na(df)
df.columns

Index(['index', 'date_block_num', 'shop_id', 'item_id', 'id', 'item_cnt_month',
       'city_code', 'item_category_id', 'type_code', 'subtype_code',
       'item_price', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12',
       'lag_3_roll_avg_3', 'lag_3_roll_avg_6', 'lag_12_roll_avg_6',
       'price_shift_1', 'price_shift_2', 'price_shift_3', 'price_shift_6',
       'price_shift_12', 'date_cnt_lag_1', 'date_item_lag_1',
       'date_item_lag_2', 'date_item_lag_3', 'date_item_lag_6',
       'date_item_lag_12', 'date_shop_lag_1', 'date_shop_lag_2',
       'date_shop_lag_3', 'date_shop_lag_6', 'date_shop_lag_12',
       'date_cat_lag_1'],
      dtype='object')

In [6]:
#group = ['item_id','date_block_num']
#df.sort_values(by = group, inplace = True)
#df
#df['mon_item_avg_1'] = groupby_shift(df, 'item_cnt_month',  'item_id', 1)

In [7]:
'''
df['price_shift_1'] = groupby_shift(df, 'item_price', 'id', 1)
df['price_shift_2'] = groupby_shift(df, 'item_price', 'id', 2)
df['price_shift_3'] = groupby_shift(df, 'item_price', 'id', 3)
df['price_shift_6'] = groupby_shift(df, 'item_price', 'id', 6)
df['price_shift_12'] = groupby_shift(df, 'item_price', 'id', 12)
'''

"\ndf['price_shift_1'] = groupby_shift(df, 'item_price', 'id', 1)\ndf['price_shift_2'] = groupby_shift(df, 'item_price', 'id', 2)\ndf['price_shift_3'] = groupby_shift(df, 'item_price', 'id', 3)\ndf['price_shift_6'] = groupby_shift(df, 'item_price', 'id', 6)\ndf['price_shift_12'] = groupby_shift(df, 'item_price', 'id', 12)\n"

In [8]:

'''
def recode_na(df, cols):
    #recode na value by grouped average
    #
    for i in tqdm(cols):
        df[i] = df[i].transform(lambda x: x.fillna(x.median()))
    return df

colz = ['shift_1', 'shift_2', 'shift_3', 'shift_6', 'shift_12',
       'shift_3_roll_avg_3', 'shift_3_roll_avg_6', 'shift_12_roll_avg_6',
       'price_lag_1', 'price_lag_2', 'price_lag_3', 'price_lag_6',
       'price_lag_12']

df = recode_na(df, colz)
'''
    
df['month'] = df['date_block_num'] % 12
df = df[df['date_block_num'] > 11]
print('data has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

data has 6639294 rows and 37 columns


In [9]:
df.columns

Index(['index', 'date_block_num', 'shop_id', 'item_id', 'id', 'item_cnt_month',
       'city_code', 'item_category_id', 'type_code', 'subtype_code',
       'item_price', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12',
       'lag_3_roll_avg_3', 'lag_3_roll_avg_6', 'lag_12_roll_avg_6',
       'price_shift_1', 'price_shift_2', 'price_shift_3', 'price_shift_6',
       'price_shift_12', 'date_cnt_lag_1', 'date_item_lag_1',
       'date_item_lag_2', 'date_item_lag_3', 'date_item_lag_6',
       'date_item_lag_12', 'date_shop_lag_1', 'date_shop_lag_2',
       'date_shop_lag_3', 'date_shop_lag_6', 'date_shop_lag_12',
       'date_cat_lag_1', 'month'],
      dtype='object')

In [10]:
features = ['date_block_num',
            'month',
            'shop_id',
            'item_id',
            'city_code', 
            'item_category_id', 
            'type_code', 
            'subtype_code',
            'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12',
       'price_shift_1', 'price_shift_2', 'price_shift_3', 'price_shift_6',
       'price_shift_12', 'date_cnt_lag_1', 'date_item_lag_1',
       'date_item_lag_2', 'date_item_lag_3', 'date_item_lag_6',
       'date_item_lag_12', 'date_shop_lag_1', 'date_shop_lag_2',
       'date_shop_lag_3', 'date_shop_lag_6', 'date_shop_lag_12',
       'date_cat_lag_1']
cat_features = ['month', 'shop_id','item_id','city_code', 'item_category_id', 'type_code', 'subtype_code']

https://catboost.ai/docs/concepts/parameter-tuning.html
https://catboost.ai/docs/concepts/python-reference_catboostregressor.html

In [11]:
#test_pred = train_lightgbm(df)
test_pred_cat = train_catboost(df)

Training fold 1


0:	learn: 1.1751902	test: 1.1571493	best: 1.1571493 (0)	total: 2.28s	remaining: 57m 3s
50:	learn: 0.8390499	test: 0.9663702	best: 0.9663702 (50)	total: 1m 26s	remaining: 40m 56s
100:	learn: 0.7660659	test: 0.9413103	best: 0.9413103 (100)	total: 2m 45s	remaining: 38m 18s
150:	learn: 0.7462132	test: 0.9314054	best: 0.9314054 (150)	total: 4m 17s	remaining: 38m 21s
200:	learn: 0.7348245	test: 0.9309967	best: 0.9305658 (185)	total: 5m 47s	remaining: 37m 26s
250:	learn: 0.7263604	test: 0.9297506	best: 0.9293779 (239)	total: 7m 18s	remaining: 36m 21s
Stopped by overfitting detector  (28 iterations wait)

bestTest = 0.9293779142
bestIteration = 239

Shrink model to first 240 iterations.
--------------------------------------------------


Training fold 2


0:	learn: 1.1655876	test: 1.2838766	best: 1.2838766 (0)	total: 3.05s	remaining: 1h 16m 14s
50:	learn: 0.8421455	test: 1.0593893	best: 1.0593893 (50)	total: 1m 59s	remaining: 56m 40s
100:	learn: 0.7795154	test: 1.0324092	best: 1.0324092 (100)	total: 4m 24s	remaining: 1h 1m 9s
150:	learn: 0.7579704	test: 1.0234693	best: 1.0234131 (149)	total: 7m 4s	remaining: 1h 3m 15s
200:	learn: 0.7466644	test: 1.0169183	best: 1.0168365 (199)	total: 9m 41s	remaining: 1h 2m 40s
250:	learn: 0.7385426	test: 1.0171915	best: 1.0163283 (233)	total: 12m 16s	remaining: 1h 1m 4s


KeyboardInterrupt: 

In [ ]:
submission = make_output(test_pred_cat)

In [ ]:
submission['item_cnt_month'].isna().value_counts()

In [ ]:
len(submission)

```
Index               67512848
date                67512848
date_block_num       8439106
shop_id              8439106
item_id             16878212
item_price          33756424
item_cnt_day         8439106
city_code            8439106
item_category_id     8439106
type_code            8439106
subtype_code         8439106
lag_1               67512848
dtype: int64
```

```
Training fold 1
Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.
0:	learn: 1.1757494	test: 1.1575727	best: 1.1575727 (0)	total: 1.43s	remaining: 35m 40s
50:	learn: 0.8640184	test: 0.9677260	best: 0.9677260 (50)	total: 48.5s	remaining: 22m 57s
100:	learn: 0.7881810	test: 0.9398471	best: 0.9398471 (100)	total: 1m 40s	remaining: 23m 10s
150:	learn: 0.7679923	test: 0.9313066	best: 0.9313066 (150)	total: 2m 47s	remaining: 24m 53s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9302290542
bestIteration = 158

Shrink model to first 159 iterations.
--------------------------------------------------


Training fold 2
Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.
0:	learn: 1.1660820	test: 1.2842634	best: 1.2842634 (0)	total: 2.38s	remaining: 59m 29s
50:	learn: 0.8826853	test: 1.0427823	best: 1.0427823 (50)	total: 1m 40s	remaining: 47m 28s
100:	learn: 0.7966804	test: 1.0309959	best: 1.0309959 (100)	total: 2m 51s	remaining: 39m 35s
150:	learn: 0.7745665	test: 1.0159513	best: 1.0159513 (150)	total: 4m 40s	remaining: 41m 48s
200:	learn: 0.7637740	test: 1.0119380	best: 1.0118339 (199)	total: 6m 27s	remaining: 41m 42s
250:	learn: 0.7561012	test: 1.0092657	best: 1.0092657 (250)	total: 8m 26s	remaining: 42m 1s
300:	learn: 0.7493929	test: 1.0079956	best: 1.0075532 (291)	total: 10m 20s	remaining: 41m 11s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.007553196
bestIteration = 291

Shrink model to first 292 iterations.
--------------------------------------------------


Training fold 3
Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.
0:	learn: 1.2064768	test: 1.0777938	best: 1.0777938 (0)	total: 3.54s	remaining: 1h 28m 33s
50:	learn: 0.9318607	test: 0.9035508	best: 0.9035508 (50)	total: 2m 22s	remaining: 1h 7m 20s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9035508379
bestIteration = 50

Shrink model to first 51 iterations.
--------------------------------------------------


Our out of folds rmse is 1.0239
```

In [3]:
for i in range(2, 11):
    print(i/100)

0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
